In [ ]:
%load_ext autoreload
%autoreload 2
from lewidi_lib import configure_pandas_display, enable_logging

configure_pandas_display()
enable_logging()


In [ ]:
from lewidi_lib import list_preds


pdf = list_preds().query(
    "(template_id == '60' or template_id == '62') and exists and dataset == 'CSC' and run_name == '1000ex_10loops' and model_id.str.contains('14B')"
)
pdf

In [ ]:
from lewidi_lib import (
    assign_cols_perf_metrics,
    join_dataset,
    load_listof_parquets,
    process_rdf,
)

task = "soft-label"
rdf = load_listof_parquets(pdf["preds_file"])
rdf = process_rdf(
    rdf, task=task, discard_invalid_pred=True, response_contains_steps=True
)
joint_df = join_dataset(rdf, task=task)
joint_df = assign_cols_perf_metrics(joint_df, task=task)

In [ ]:
s1 = set(joint_df.query("template_id == '60'")["dataset_idx"])
s2 = set(joint_df.query("template_id == '62'")["dataset_idx"])
common = s1.intersection(s2)
len(common)

In [ ]:
from lewidi_lib import bootstrap_avg


joint_df.query("dataset_idx.isin(@common)").groupby("template_id")["ws_loss"].agg(bootstrap_avg)

In [ ]:
from lewidi_lib import compute_average_baseline_and_assing_perf_metrics


model_avg_df = compute_average_baseline_and_assing_perf_metrics(rdf)
model_avg_df.query("dataset_idx.isin(@common)").groupby("template_id")["ws_loss"].agg(bootstrap_avg)